<img src="images/dask_horizontal.svg" align="right" width="30%">

# Bag: Parallel Lists for semi-structured data

Dask-bag excels in processing data that can be represented as a sequence of arbitrary inputs. We'll refer to this as "messy" data, because it can contain complex nested structures, missing fields, mixtures of data types, etc. The *functional* programming style fits very nicely with standard Python iteration, such as can be found in the `itertools` module.

Messy data is often encountered at the beginning of data processing pipelines when large volumes of raw data are first consumed. The initial set of data might be JSON, CSV, XML, or any other format that does not enforce strict structure and datatypes.
For this reason, the initial data massaging and processing is often done with Python `list`s, `dict`s, and `set`s.

These core data structures are optimized for general-purpose storage and processing.  Adding streaming computation with iterators/generator expressions or libraries like `itertools` or [`toolz`](https://toolz.readthedocs.io/en/latest/) let us process large volumes in a small space.  If we combine this with parallel processing then we can churn through a fair amount of data.

Dask.bag is a high level Dask collection to automate common workloads of this form.  In a nutshell

    dask.bag = map, filter, toolz + parallel execution
    
**Related Documentation**

* [Bag documentation](https://docs.dask.org/en/latest/bag.html)
* [Bag screencast](https://youtu.be/-qIiJ1XtSv0)
* [Bag API](https://docs.dask.org/en/latest/bag-api.html)
* [Bag examples](https://examples.dask.org/bag.html)

## Create data

In [1]:
%run prep.py -d accounts

## Setup

Again, we'll use the distributed scheduler. Schedulers will be explained in depth [later](05_distributed.ipynb).

In [2]:
from dask.distributed import Client

client = Client(n_workers=4)

## Creation

You can create a `Bag` from a Python sequence, from files, from data on S3, etc.
We demonstrate using `.take()` to show elements of the data. (Doing `.take(1)` results in a tuple with one element)

Note that the data are partitioned into blocks, and there are many items per block. In the first example, the two partitions contain five elements each, and in the following two, each file is partitioned into one or more bytes blocks.

In [3]:
# each element is an integer
import dask.bag as db
b = db.from_sequence([1, 2, 3, 4, 5, 6, 7, 8, 9, 10], npartitions=2)
b.take(3)

(1, 2, 3)

In [4]:
# each element is a text file, where each line is a JSON object
# note that the compression is handled automatically
import os
b = db.read_text(os.path.join('data', 'accounts.*.json.gz'))
b.take(1)

('{"id": 0, "name": "Victor", "transactions": [{"transaction-id": 263, "amount": 8733}, {"transaction-id": 357, "amount": 8576}, {"transaction-id": 456, "amount": 9339}, {"transaction-id": 591, "amount": 8432}, {"transaction-id": 661, "amount": 7560}, {"transaction-id": 756, "amount": 9293}, {"transaction-id": 808, "amount": 7848}, {"transaction-id": 859, "amount": 8172}, {"transaction-id": 940, "amount": 8280}, {"transaction-id": 1417, "amount": 8667}, {"transaction-id": 1566, "amount": 8641}, {"transaction-id": 1897, "amount": 8323}, {"transaction-id": 2114, "amount": 8322}, {"transaction-id": 2176, "amount": 8178}, {"transaction-id": 2197, "amount": 8725}, {"transaction-id": 2198, "amount": 9324}, {"transaction-id": 2389, "amount": 8350}, {"transaction-id": 2439, "amount": 9230}, {"transaction-id": 2510, "amount": 9154}, {"transaction-id": 2568, "amount": 7961}, {"transaction-id": 2665, "amount": 7982}, {"transaction-id": 2975, "amount": 8327}, {"transaction-id": 3200, "amount": 789

In [5]:
# Edit sources.py to configure source locations
import sources
sources.bag_url

's3://dask-data/nyc-taxi/2015/yellow_tripdata_2015-01.csv'

In [6]:
# Requires `s3fs` library
# each partition is a remote CSV text file
b = db.read_text(sources.bag_url,
                 storage_options={'anon': True})
b.take(1)

('VendorID,tpep_pickup_datetime,tpep_dropoff_datetime,passenger_count,trip_distance,pickup_longitude,pickup_latitude,RateCodeID,store_and_fwd_flag,dropoff_longitude,dropoff_latitude,payment_type,fare_amount,extra,mta_tax,tip_amount,tolls_amount,improvement_surcharge,total_amount\n',)

## Manipulation

`Bag` objects hold the standard functional API found in projects like the Python standard library, `toolz`, or `pyspark`, including `map`, `filter`, `groupby`, etc..

Operations on `Bag` objects create new bags.  Call the `.compute()` method to trigger execution, as we saw for `Delayed` objects.  

In [7]:
def is_even(n):
    return n % 2 == 0

b = db.from_sequence([1, 2, 3, 4, 5, 6, 7, 8, 9, 10])
c = b.filter(is_even).map(lambda x: x ** 2)
c

dask.bag<lambda, npartitions=10>

In [8]:
# blocking form: wait for completion (which is very fast in this case)
c.compute()

[4, 16, 36, 64, 100]

### Example: Accounts JSON data

We've created a fake dataset of gzipped JSON data in your data directory.  This is like the example used in the `DataFrame` example we will see later, except that it has bundled up all of the entires for each individual `id` into a single record.  This is similar to data that you might collect off of a document store database or a web API.

Each line is a JSON encoded dictionary with the following keys

*  id: Unique identifier of the customer
*  name: Name of the customer
*  transactions: List of `transaction-id`, `amount` pairs, one for each transaction for the customer in that file

In [9]:
filename = os.path.join('data', 'accounts.*.json.gz')
lines = db.read_text(filename)
lines.take(3)

('{"id": 0, "name": "Victor", "transactions": [{"transaction-id": 263, "amount": 8733}, {"transaction-id": 357, "amount": 8576}, {"transaction-id": 456, "amount": 9339}, {"transaction-id": 591, "amount": 8432}, {"transaction-id": 661, "amount": 7560}, {"transaction-id": 756, "amount": 9293}, {"transaction-id": 808, "amount": 7848}, {"transaction-id": 859, "amount": 8172}, {"transaction-id": 940, "amount": 8280}, {"transaction-id": 1417, "amount": 8667}, {"transaction-id": 1566, "amount": 8641}, {"transaction-id": 1897, "amount": 8323}, {"transaction-id": 2114, "amount": 8322}, {"transaction-id": 2176, "amount": 8178}, {"transaction-id": 2197, "amount": 8725}, {"transaction-id": 2198, "amount": 9324}, {"transaction-id": 2389, "amount": 8350}, {"transaction-id": 2439, "amount": 9230}, {"transaction-id": 2510, "amount": 9154}, {"transaction-id": 2568, "amount": 7961}, {"transaction-id": 2665, "amount": 7982}, {"transaction-id": 2975, "amount": 8327}, {"transaction-id": 3200, "amount": 789

Our data comes out of the file as lines of text. Notice that file decompression happened automatically. We can make this data look more reasonable by mapping the `json.loads` function onto our bag.

In [10]:
import json
js = lines.map(json.loads)
# take: inspect first few elements
js.take(3)

({'id': 0,
  'name': 'Victor',
  'transactions': [{'transaction-id': 263, 'amount': 8733},
   {'transaction-id': 357, 'amount': 8576},
   {'transaction-id': 456, 'amount': 9339},
   {'transaction-id': 591, 'amount': 8432},
   {'transaction-id': 661, 'amount': 7560},
   {'transaction-id': 756, 'amount': 9293},
   {'transaction-id': 808, 'amount': 7848},
   {'transaction-id': 859, 'amount': 8172},
   {'transaction-id': 940, 'amount': 8280},
   {'transaction-id': 1417, 'amount': 8667},
   {'transaction-id': 1566, 'amount': 8641},
   {'transaction-id': 1897, 'amount': 8323},
   {'transaction-id': 2114, 'amount': 8322},
   {'transaction-id': 2176, 'amount': 8178},
   {'transaction-id': 2197, 'amount': 8725},
   {'transaction-id': 2198, 'amount': 9324},
   {'transaction-id': 2389, 'amount': 8350},
   {'transaction-id': 2439, 'amount': 9230},
   {'transaction-id': 2510, 'amount': 9154},
   {'transaction-id': 2568, 'amount': 7961},
   {'transaction-id': 2665, 'amount': 7982},
   {'transaction-

### Basic Queries

Once we parse our JSON data into proper Python objects (`dict`s, `list`s, etc.) we can perform more interesting queries by creating small Python functions to run on our data.

In [11]:
# filter: keep only some elements of the sequence
js.filter(lambda record: record['name'] == 'Alice').take(5)

({'id': 13,
  'name': 'Alice',
  'transactions': [{'transaction-id': 30, 'amount': 605}]},
 {'id': 38,
  'name': 'Alice',
  'transactions': [{'transaction-id': 8, 'amount': 467},
   {'transaction-id': 468, 'amount': 459},
   {'transaction-id': 901, 'amount': 419},
   {'transaction-id': 1309, 'amount': 378},
   {'transaction-id': 1684, 'amount': 571},
   {'transaction-id': 1870, 'amount': 445},
   {'transaction-id': 2257, 'amount': 523},
   {'transaction-id': 2703, 'amount': 472},
   {'transaction-id': 4083, 'amount': 503},
   {'transaction-id': 4570, 'amount': 504},
   {'transaction-id': 5140, 'amount': 460},
   {'transaction-id': 5178, 'amount': 419},
   {'transaction-id': 7037, 'amount': 587},
   {'transaction-id': 7359, 'amount': 453},
   {'transaction-id': 7875, 'amount': 508},
   {'transaction-id': 8421, 'amount': 520},
   {'transaction-id': 8749, 'amount': 406},
   {'transaction-id': 8848, 'amount': 492}]},
 {'id': 59,
  'name': 'Alice',
  'transactions': [{'transaction-id': 423,

In [12]:
def count_transactions(d):
    return {'name': d['name'], 'count': len(d['transactions'])}

# map: apply a function to each element
(js.filter(lambda record: record['name'] == 'Alice')
   .map(count_transactions)
   .take(5))

({'name': 'Alice', 'count': 1},
 {'name': 'Alice', 'count': 18},
 {'name': 'Alice', 'count': 8},
 {'name': 'Alice', 'count': 17},
 {'name': 'Alice', 'count': 73})

In [13]:
# pluck: select a field, as from a dictionary, element[field]
(js.filter(lambda record: record['name'] == 'Alice')
   .map(count_transactions)
   .pluck('count')
   .take(5))

(1, 18, 8, 17, 73)

In [14]:
# Average number of transactions for all of the Alice entries
(js.filter(lambda record: record['name'] == 'Alice')
   .map(count_transactions)
   .pluck('count')
   .mean()
   .compute())

32.475103734439834

### Use `flatten` to de-nest

In the example below we see the use of `.flatten()` to flatten results.  We compute the average amount for all transactions for all Alices.

In [15]:
js.filter(lambda record: record['name'] == 'Alice').pluck('transactions').take(3)

([{'transaction-id': 30, 'amount': 605}],
 [{'transaction-id': 8, 'amount': 467},
  {'transaction-id': 468, 'amount': 459},
  {'transaction-id': 901, 'amount': 419},
  {'transaction-id': 1309, 'amount': 378},
  {'transaction-id': 1684, 'amount': 571},
  {'transaction-id': 1870, 'amount': 445},
  {'transaction-id': 2257, 'amount': 523},
  {'transaction-id': 2703, 'amount': 472},
  {'transaction-id': 4083, 'amount': 503},
  {'transaction-id': 4570, 'amount': 504},
  {'transaction-id': 5140, 'amount': 460},
  {'transaction-id': 5178, 'amount': 419},
  {'transaction-id': 7037, 'amount': 587},
  {'transaction-id': 7359, 'amount': 453},
  {'transaction-id': 7875, 'amount': 508},
  {'transaction-id': 8421, 'amount': 520},
  {'transaction-id': 8749, 'amount': 406},
  {'transaction-id': 8848, 'amount': 492}],
 [{'transaction-id': 423, 'amount': 2923},
  {'transaction-id': 3121, 'amount': 3184},
  {'transaction-id': 3349, 'amount': 3246},
  {'transaction-id': 4260, 'amount': 3338},
  {'transacti

In [16]:
(js.filter(lambda record: record['name'] == 'Alice')
   .pluck('transactions')
   .flatten()
   .take(3))

({'transaction-id': 30, 'amount': 605},
 {'transaction-id': 8, 'amount': 467},
 {'transaction-id': 468, 'amount': 459})

In [17]:
(js.filter(lambda record: record['name'] == 'Alice')
   .pluck('transactions')
   .flatten()
   .pluck('amount')
   .take(3))

(605, 467, 459)

In [18]:
(js.filter(lambda record: record['name'] == 'Alice')
   .pluck('transactions')
   .flatten()
   .pluck('amount')
   .mean()
   .compute())

1356.714048425222

### Groupby and Foldby

Often we want to group data by some function or key.  We can do this either with the `.groupby` method, which is straightforward but forces a full shuffle of the data (expensive) or with the harder-to-use but faster `.foldby` method, which does a streaming combined groupby and reduction.

*  `groupby`:  Shuffles data so that all items with the same key are in the same key-value pair
*  `foldby`:  Walks through the data accumulating a result per key

*Note: the full groupby is particularly bad. In actual workloads you would do well to use `foldby` or switch to `DataFrame`s if possible.*

### `groupby`

Groupby collects items in your collection so that all items with the same value under some function are collected together into a key-value pair.

In [19]:
b = db.from_sequence(['Alice', 'Bob', 'Charlie', 'Dan', 'Edith', 'Frank'])
b.groupby(len).compute()  # names grouped by length

[(7, ['Charlie']), (3, ['Bob', 'Dan']), (5, ['Alice', 'Edith', 'Frank'])]

In [20]:
b = db.from_sequence(list(range(10)))
b.groupby(lambda x: x % 2).compute()

[(0, [0, 2, 4, 6, 8]), (1, [1, 3, 5, 7, 9])]

In [21]:
b.groupby(lambda x: x % 2).starmap(lambda k, v: (k, max(v))).compute()

[(0, 8), (1, 9)]

### `foldby`

Foldby can be quite odd at first.  It is similar to the following functions from other libraries:

*  [`toolz.reduceby`](http://toolz.readthedocs.io/en/latest/streaming-analytics.html#streaming-split-apply-combine)
*  [`pyspark.RDD.combineByKey`](http://abshinn.github.io/python/apache-spark/2014/10/11/using-combinebykey-in-apache-spark/)

When using `foldby` you provide 

1.  A key function on which to group elements
2.  A binary operator such as you would pass to `reduce` that you use to perform reduction per each group
3.  A combine binary operator that can combine the results of two `reduce` calls on different parts of your dataset.

Your reduction must be associative.  It will happen in parallel in each of the partitions of your dataset.  Then all of these intermediate results will be combined by the `combine` binary operator.

In [22]:
is_even = lambda x: x % 2
b.foldby(is_even, binop=max, combine=max).compute()

[(0, 8), (1, 9)]

### Example with account data

We find the number of people with the same name.

In [23]:
%%time
# Warning, this one takes a while...
result = js.groupby(lambda item: item['name']).starmap(lambda k, v: (k, len(v))).compute()
print(sorted(result))

[('Alice', 107), ('Alice', 119), ('Alice', 122), ('Alice', 134), ('Bob', 44), ('Bob', 48), ('Bob', 52), ('Bob', 56), ('Charlie', 110), ('Charlie', 120), ('Charlie', 130), ('Charlie', 140), ('Dan', 88), ('Dan', 96), ('Dan', 104), ('Dan', 112), ('Edith', 66), ('Edith', 72), ('Edith', 78), ('Edith', 84), ('Frank', 91), ('Frank', 98), ('Frank', 161), ('George', 107), ('George', 114), ('George', 122), ('George', 132), ('Hannah', 175), ('Hannah', 190), ('Hannah', 206), ('Hannah', 221), ('Ingrid', 110), ('Ingrid', 119), ('Ingrid', 130), ('Ingrid', 140), ('Jerry', 139), ('Jerry', 151), ('Jerry', 163), ('Jerry', 174), ('Kevin', 77), ('Kevin', 84), ('Kevin', 91), ('Kevin', 98), ('Laura', 144), ('Laura', 156), ('Laura', 300), ('Michael', 108), ('Michael', 116), ('Michael', 127), ('Michael', 140), ('Norbert', 121), ('Norbert', 132), ('Norbert', 143), ('Norbert', 154), ('Oliver', 110), ('Oliver', 116), ('Oliver', 126), ('Oliver', 135), ('Patricia', 89), ('Patricia', 96), ('Patricia', 106), ('Patric

In [24]:
%%time
# This one is comparatively fast and produces the same result.
from operator import add
def incr(tot, _):
    return tot+1

result = js.foldby(key='name', 
                   binop=incr, 
                   initial=0, 
                   combine=add, 
                   combine_initial=0).compute()
print(sorted(result))

[('Alice', 482), ('Bob', 200), ('Charlie', 500), ('Dan', 400), ('Edith', 300), ('Frank', 350), ('George', 475), ('Hannah', 792), ('Ingrid', 499), ('Jerry', 627), ('Kevin', 350), ('Laura', 600), ('Michael', 491), ('Norbert', 550), ('Oliver', 487), ('Patricia', 404), ('Quinn', 528), ('Ray', 448), ('Sarah', 626), ('Tim', 355), ('Ursula', 245), ('Victor', 400), ('Wendy', 390), ('Xavier', 642), ('Yvonne', 532), ('Zelda', 398)]
CPU times: user 127 ms, sys: 16.9 ms, total: 144 ms
Wall time: 597 ms


### Exercise: compute total amount per name

We want to groupby (or foldby) the `name` key, then add up the all of the amounts for each name.

Steps

1.  Create a small function that, given a dictionary like 

        {'name': 'Alice', 'transactions': [{'amount': 1, 'id': 123}, {'amount': 2, 'id': 456}]}
        
    produces the sum of the amounts, e.g. `3`
    
2.  Slightly change the binary operator of the `foldby` example above so that the binary operator doesn't count the number of entries, but instead accumulates the sum of the amounts.

In [25]:
# Your code here...

## DataFrames

For the same reasons that Pandas is often faster than pure Python, `dask.dataframe` can be faster than `dask.bag`.  We will work more with DataFrames later, but from for the bag point of view, they are frequently the end-point of the "messy" part of data ingestion—once the data can be made into a data-frame, then complex split-apply-combine logic will become much more straight-forward and efficient.

You can transform a bag with a simple tuple or flat dictionary structure into a `dask.dataframe` with the `to_dataframe` method.

In [26]:
df1 = js.to_dataframe()
df1.head()

,id,name,transactions
0,0,Victor,"[{'transaction-id': 263, 'amount': 8733}, {'tr..."
1,1,Norbert,"[{'transaction-id': 1811, 'amount': 612}, {'tr..."
2,3,Patricia,"[{'transaction-id': 24, 'amount': -153}, {'tra..."
3,4,Laura,"[{'transaction-id': 115, 'amount': 3181}, {'tr..."
4,5,Jerry,"[{'transaction-id': 105, 'amount': 1872}, {'tr..."


This now looks like a well-defined DataFrame, and we can apply Pandas-like computations to it efficiently.

Using a Dask DataFrame, how long does it take to do our prior computation of numbers of people with the same name?  It turns out that `dask.dataframe.groupby()` beats `dask.bag.groupby()` more than an order of magnitude; but it still cannot match `dask.bag.foldby()` for this case.

In [27]:
%time df1.groupby('name').id.count().compute().head()

CPU times: user 183 ms, sys: 6.19 ms, total: 189 ms
Wall time: 1.62 s


name
Alice      482
Bob        200
Charlie    500
Dan        400
Edith      300
Name: id, dtype: int64

### Denormalization

This DataFrame format is less-than-optimal because the `transactions` column is filled with nested data so Pandas has to revert to `object` dtype, which is quite slow in Pandas.  Ideally we want to transform to a dataframe only after we have flattened our data so that each record is a single `int`, `string`, `float`, etc..

In [28]:
def denormalize(record):
    # returns a list for every nested item, each transaction of each person
    return [{'id': record['id'], 
             'name': record['name'], 
             'amount': transaction['amount'], 
             'transaction-id': transaction['transaction-id']}
            for transaction in record['transactions']]

transactions = js.map(denormalize).flatten()
transactions.take(3)

({'id': 0, 'name': 'Victor', 'amount': 8733, 'transaction-id': 263},
 {'id': 0, 'name': 'Victor', 'amount': 8576, 'transaction-id': 357},
 {'id': 0, 'name': 'Victor', 'amount': 9339, 'transaction-id': 456})

In [29]:
df = transactions.to_dataframe()
df.head()

,id,name,amount,transaction-id
0,0,Victor,8733,263
1,0,Victor,8576,357
2,0,Victor,9339,456
3,0,Victor,8432,591
4,0,Victor,7560,661


In [30]:
%%time
# number of transactions per name
# note that the time here includes the data load and ingestion
df.groupby('name')['transaction-id'].count().compute()

CPU times: user 170 ms, sys: 6.05 ms, total: 176 ms
Wall time: 1.38 s


name
Alice       15653
Bob          4632
Charlie     29152
Dan         16864
Edith        8697
Frank       16002
George      10003
Hannah      26605
Ingrid      23008
Jerry       29980
Kevin       18291
Laura       29101
Michael     21620
Norbert     22259
Oliver      16627
Patricia    20293
Quinn       20768
Ray         14014
Sarah       30872
Tim         13257
Ursula       6935
Victor      18961
Wendy       22861
Xavier      36352
Yvonne      18380
Zelda        8813
Name: transaction-id, dtype: int64

## Limitations

Bags provide very general computation (any Python function.)  This generality
comes at cost.  Bags have the following known limitations

1.  Bag operations tend to be slower than array/dataframe computations in the
    same way that Python tends to be slower than NumPy/Pandas
2.  ``Bag.groupby`` is slow.  You should try to use ``Bag.foldby`` if possible.
    Using ``Bag.foldby`` requires more thought. Even better, consider creating
    a normalised dataframe.

## Learn More

* [Bag documentation](https://docs.dask.org/en/latest/bag.html)
* [Bag screencast](https://youtu.be/-qIiJ1XtSv0)
* [Bag API](https://docs.dask.org/en/latest/bag-api.html)
* [Bag examples](https://examples.dask.org/bag.html)

## Shutdown

In [31]:
client.shutdown()